# Reliability calculations with FORM

In this example, we will demonstrate how to perform reliability calculations using the First Order Reliability Method (FORM).

### Define model

First, let's import the necessary package:

In [25]:
from streams import *

Next, we define a simple limit state function: 

$Z = 1.9 - (a+b)$

This is a linear model involving two variables, $a$ and $b$.

In [26]:
def linear(a, b):
    return 1.9 - (a+b)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [27]:
project = ReliabilityProject()
project.model = linear

We assume that variables $a$ and $b$ are uniformly distributed over the interval $[-1, 1]$. This is defined as follows:

In [28]:
project.variables["a"].distribution = DistributionType.uniform
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = DistributionType.uniform
project.variables["b"].minimum = -1
project.variables["b"].maximum = 1

### Define reliability method

Next, we define the reliability method: `form`. We choose the calculation settings: `relaxation_factor`, `maximum_iterations` and `variation_coefficient`. 

In [29]:
project.settings.reliability_method = ReliabilityMethod.form

project.settings.relaxation_factor = 0.75
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

We can also define the start method, with the following options available: `ray_search`, `one`, `fixed_value`, `sensitivity_search`, and `sphere_search`. If the fixed_value option is selected, a start value must be specified for each variable.

In [30]:
# fixed_value
project.settings.start_method = StartMethod.fixed_value
project.settings.stochast_settings["a"].start_value = 1.2
project.settings.stochast_settings["b"].start_value = 2.4

# in this example we choose the ray_search method
project.settings.start_method = StartMethod.ray_search

### Perform calculations

We use `project.run()` to execute the reliability analysis:

In [31]:
project.run()

The results are written to `project.design_point` and consist of:
* reliability index $\beta$
* failure probability $P_f$
* influence coefficients $\alpha$-values
* design point $x$-values
* information about the convergence of FORM


In [32]:
def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Beta = 2.085219001013316
Probability of failure = 0.018524712950179534
a: alpha = -0.31495828992212993, x = 0.4886628297566449
b: alpha = -0.9491055134226793, x = 0.9521944467204182
Not converged (convergence = 0.6708444811528529 > 0.02)
Model runs = 150


### Correlated variables

In the above example, variables $a$ and $b$ are independent. However, it is also possible to introduce a correlation between $a$ and $b$. The example below demonstrates this with a partial correlation coefficient of $0.8$.

In [33]:
project.correlation_matrix[("a", "b")] = 0.8
project.run()
read_results(project.design_point)

Beta = 2.066415710261457
Probability of failure = 0.01939462189035001
a: alpha = -0.9455959674963248, x = 0.9492981161382412
b: alpha = -0.32534330522494176, x = 0.9507674490087261
Converged (convergence = 0.002497376786762608 < 0.02)
Model runs = 15
